In [1]:
from dataclasses import dataclass
from functools import reduce
import itertools
from fractions import Fraction
import numpy as np

from music21.pitch import Pitch

In [2]:
# round to nearest n division of a whole tone. 2 is usual (1/2), 4 is quarter-tone (1/4), etc.
def round_midi_to_microtone(x, microtone=2):
    microtone //= 2

    # base midi number
    base = round(x)
    # actual deviation
    dev = x - base

    # quantized deviation
    q_dev = round((dev * microtone * 100)/100)/microtone

    # np.float everywhere, looks ugly
    return base, str(Fraction(q_dev)/2), str(f'{dev*100:.2f}')

In [3]:
def midi_to_note_name(midi_number):
    sharp_note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    flat_note_names = ['C', 'Db', 'D', 'Eb', 'E', 'F', 'Gb', 'G', 'Ab', 'A', 'Bb', 'B']
    octave = midi_number // 12 - 1
    sharp_note = sharp_note_names[midi_number % 12]
    flat_note = flat_note_names[midi_number % 12]
    
    # if sharp_note == flat_note:
    return f"{sharp_note}{octave}"
    # else:
        # return f"{sharp_note}{octave} / {flat_note}{octave}"

def note_name_to_midi(note_name):
    note_name = note_name.upper()
    # Mapping of note names to their semitone positions relative to C
    note_to_semitone = {
        'C': 0, 'C#': 1, 'Db': 1, 'D': 2, 'D#': 3, 'Eb': 3, 
        'E': 4, 'F': 5, 'F#': 6, 'Gb': 6, 'G': 7, 'G#': 8, 
        'Ab': 8, 'A': 9, 'A#': 10, 'Bb': 10, 'B': 11
    }
    
    # Extract the note and octave from the input string
    note = note_name[:-1]
    octave = int(note_name[-1])
    
    # Calculate the MIDI number
    midi_number = 12 * (octave + 1) + note_to_semitone[note]
    
    return midi_number

def midi_to_hz(midi):
    return 440 * 2**((midi - 69) / 12)

def hz_to_midi(f):
    return 69 + 12 * np.log2(f / 440)

In [4]:
# f0 is the MIDI number of the fundamental frequency
# precision = how many divisions of a whole tone. 2 is 12-edo, 4 is quarter tone, etc.
# first N overtones or just give a nth overtone?
def overtone(f0_midi, n):
    hz = midi_to_hz(f0_midi)
    result = hz_to_midi(hz * n)

    return result

def nearest_midi_note(midi):
    return round(midi)

def nearest_microtone_adjustment(midi, microtone):
    microtone //= 2
    # base midi number
    base = round(midi)
    # actual deviation
    dev = midi - base
    # quantized deviation
    q_dev = round((dev * microtone * 100)/100)/microtone

    return Fraction(q_dev)/2

def reduce_octaves(midi):
    return midi % 12

In [60]:
# cents to sibelius pitch bend value
def cents_to_pitch_bend(cents):
    if cents <= 0:
        value = (1 + cents / 200) * 64
    else:
        value = (cents / 200) * (127 - 64) + 64

    return round(value)

In [117]:
@dataclass
class MicrotonalPitch:
    midi: float
    microtones: int = 8

    def __repr__(self):
        return f'{self.name:<4} {str(self.microtone_deviation):>4} {self.cents_deviation:>3} {self.pitch_bend:>2}'
    
    def __str__(self):
        return self.__repr__()

    def __eq(self, other):
        return (self.name == other.name) and (self.microtone_deviation == other.microtone_deviation)

    @property
    def base_midi(self):
        return round(self.midi)

    @property
    def name(self):
        return midi_to_note_name(self.base_midi)

    @property
    def cents_deviation(self):
        return round(100 * (self.midi - self.base_midi))

    @property
    def pitch_bend(self):
        return cents_to_pitch_bend(self.cents_deviation)
    
    @property
    def microtone_deviation(self):
        return nearest_microtone_adjustment(self.midi, self.microtones)

In [118]:
def overtone_series(note, microtones, n_overtones):
    midi = note_name_to_midi(note)
    microtonal_pitches = []
    for i in range(1, n_overtones+1):
        o_midi = overtone(midi, i)
        microtonal_pitches.append(MicrotonalPitch(o_midi, microtones))

    return microtonal_pitches

In [119]:
# if you touch at 1-1/n, 1/n from above
def string_length_ratio_to_cents(n):
    all_points = []
    for i in range(1, n):
        f_touched_length = 1 - i/n
        # f_whole_string is 1    
        ratio_to_cents = hz_to_midi(1/f_touched_length) - hz_to_midi(1)
        all_points.append(ratio_to_cents)

    return all_points

In [120]:
@dataclass
class Overtone:
    note: MicrotonalPitch
    base: MicrotonalPitch
    n: int

    def __repr__(self):
        return f'{self.n} {self.note}'

In [121]:
def get_nth_harmonic_locations(note, n, microtones=8):
    midi_note = note_name_to_midi(note)
    intervals = string_length_ratio_to_cents(n)
    touch_points = [interval + midi_note for interval in intervals]
    touch_points = [MicrotonalPitch(tp, microtones) for tp in touch_points]

    # sounding = [Overtone(MicrotonalPitch(overtone(midi_note, n), microtones), MicrotonalPitch(note, microtones), n)]

    return touch_points

In [122]:
def is_any_item_present(needle_list, haystack_lists):
    return any(item in sublist for sublist in haystack_lists for item in needle_list)

def is_item_present(needle, haystack_lists):
    return any(needle in sublist for sublist in haystack_lists)

In [123]:
def get_first_n_harmonic_locations(note, n, microtones=8):
    locations_all = []
    for harm in range(2, n+1):
        locations_all.append(get_nth_harmonic_locations(note, harm, microtones))

    # clean up. remove all touch points that came up in a lower harmoinic
    filtered = []
    for idx, locs in enumerate(locations_all[::-1]):
        filtered.append([toch for toch in locs if not is_item_present(toch, locations_all[:-idx-1])])

    filtered = filtered[::-1]
    
    overts = overtone_series(note, microtones, n)[1:]
    sounding = [Overtone(ov, MicrotonalPitch(note, microtones), idx+2) for idx, ov in enumerate(overts)]

    # returns pairs of (list touch points) and (sounding overtone)
    return list(zip(filtered, sounding))

In [124]:
# string = 'c4'
for string in ['c3', 'g3', 'd4', 'a4']:
    base_note = 'e4'
    
    c_touch_points = get_first_n_harmonic_locations(string, 8)
    current_overtones = overtone_series(base_note, 8, 35)
    # pair overtone number with notes
    current_overtones = [(idx+1, ov) for idx, ov in enumerate(current_overtones)]
    # only odd overtones
    current_overtones = [ov for ov in current_overtones if ov[0] % 2 == 1]
    # current_overtones[n] = (overtone_number, note)
    # c_touchpoints[0][1].note = sounding_note
    
    filtered = []
    for tp in c_touch_points:
        for ov in current_overtones:
            if abs((tp[1].note.midi % 12) - (ov[1].midi % 12)) <= 0.20:
                filtered.append((tp, ov))

    filtered = sorted(filtered, key=lambda x: x[1][1].midi)
    
    print(f'\n\n{string=}, {base_note=}\n')
    printed = []
    for fi in filtered:
        tp, ov = fi
        if ov[0] not in printed:
            print(f'{ov[0]:<2} {ov[1]}')
            print('-------------')
            printed.append(ov[0])
        
        print(f'{tp[1].n:<2} {tp[0]}')
    print('___________________')



string='c3', base_note='e4'

1  E4      0   0 64
-------------
5  [E3   -1/8 -14 60, A3   -1/8 -16 59, E4   -1/8 -14 60, E5   -1/8 -14 60]
11 A#7  -1/4 -49 48
-------------
7  [D#3  -1/8 -33 53, F#3  -1/8 -17 59, A#3  -1/8 -31 54, D#4  -1/8 -33 53, A#4  -1/8 -31 54, A#5  -1/8 -31 54]
19 G8      0  -2 63
-------------
3  [G3      0   2 65, G4      0   2 65]
6  [D#3   1/8  16 69, G5      0   2 65]
___________________


string='g3', base_note='e4'

3  B5      0   2 65
-------------
5  [B3   -1/8 -14 60, E4   -1/8 -16 59, B4   -1/8 -14 60, B5   -1/8 -14 60]
19 G8      0  -2 63
-------------
2  [G4      0   0 64]
4  [C4      0  -2 63, G5      0   0 64]
8  [A3    1/8  31 74, D#4   1/8  14 68, C5      0  -2 63, G6      0   0 64]
33 F9   -1/4 -47 49
-------------
7  [A#3  -1/8 -33 53, C#4  -1/8 -17 59, F4   -1/8 -31 54, A#4  -1/8 -33 53, F5   -1/8 -31 54, F6   -1/8 -31 54]
___________________


string='d4', base_note='e4'

9  F#7     0   4 65
-------------
5  [F#4  -1/8 -14 60, B4   -1/8 -16

In [125]:
touch_points = get_first_n_harmonic_locations('e5', 8)

filtered = []
for tps in touch_points:
    hophop = []
    for tp in tps[0]:
        if abs(tp.cents_deviation) <= 14:
            hophop.append(tp)
    if hophop:
        lolo = (hophop, tps[1])
        filtered.append(lolo)

for tps in filtered:
    print(tps)

([E6      0   0 64], 2 E6      0   0 64)
([B5      0   2 65, B6      0   2 65], 3 B6      0   2 65)
([A5      0  -2 63, E7      0   0 64], 4 E7      0   0 64)
([G#5  -1/8 -14 60, G#6  -1/8 -14 60, G#7  -1/8 -14 60], 5 G#7  -1/8 -14 60)
([B7      0   2 65], 6 B7      0   2 65)
([C6    1/8  14 68, A6      0  -2 63, E8      0   0 64], 8 E8      0   0 64)


In [132]:
for note in ['c3', 'g3', 'd4', 'a4', 'e5']:
    touch_points = get_first_n_harmonic_locations(note, 8)
    print(note)
    for tp in touch_points:
        print(f'{tp[1]}')
        for x in tp[0]:
            print(f'\t{x}')

c3
2 C4      0   0 64
	C4      0   0 64
3 G4      0   2 65
	G3      0   2 65
	G4      0   2 65
4 C5      0   0 64
	F3      0  -2 63
	C5      0   0 64
5 E5   -1/8 -14 60
	E3   -1/8 -14 60
	A3   -1/8 -16 59
	E4   -1/8 -14 60
	E5   -1/8 -14 60
6 G5      0   2 65
	D#3   1/8  16 69
	G5      0   2 65
7 A#5  -1/8 -31 54
	D#3  -1/8 -33 53
	F#3  -1/8 -17 59
	A#3  -1/8 -31 54
	D#4  -1/8 -33 53
	A#4  -1/8 -31 54
	A#5  -1/8 -31 54
8 C6      0   0 64
	D3    1/8  31 74
	G#3   1/8  14 68
	F4      0  -2 63
	C6      0   0 64
g3
2 G4      0   0 64
	G4      0   0 64
3 D5      0   2 65
	D4      0   2 65
	D5      0   2 65
4 G5      0   0 64
	C4      0  -2 63
	G5      0   0 64
5 B5   -1/8 -14 60
	B3   -1/8 -14 60
	E4   -1/8 -16 59
	B4   -1/8 -14 60
	B5   -1/8 -14 60
6 D6      0   2 65
	A#3   1/8  16 69
	D6      0   2 65
7 F6   -1/8 -31 54
	A#3  -1/8 -33 53
	C#4  -1/8 -17 59
	F4   -1/8 -31 54
	A#4  -1/8 -33 53
	F5   -1/8 -31 54
	F6   -1/8 -31 54
8 G6      0   0 64
	A3    1/8  31 74
	D#4   1/8  14 68
	C5     

## Garbage below

In [ ]:
# `up` is unnecessary. they are the same as the actual sounding notes
def _harmonic_location(n, up=True):
    if up:
        n = 1 / (1 - 1/n)
    
    return 12 * np.log2(n)

def harmonic_location(n, note, microtones, up=True):
    distance = _harmonic_location(n, up)
    midi = note_name_to_midi(note)
    position = distance + midi

    return float_midi_to_microtonal_pitch(position, microtones)

In [ ]:
note = "e4"
# filter odd overtones
my_overtones = overtone_series(note, 8, 35)
for idx, val in zip(range(1, 36), my_overtones):
    if idx % 2 == 0:
        continue
    print(f'{idx:<2}: {val} - pitch bend: {cents_to_pitch_bend(val.cents_deviation)}')
    # microtonal pitch bend: {cents_to_pitch_bend(val.microtone_deviation * 200)} - actual
    # print(f'{idx:<2}: {val}')

In [ ]:
for i in range(2, 13):
    print(i, harmonic_location(i, note, 8), "    ", float_midi_to_microtonal_pitch(overtone(note_name_to_midi(note), i), 8))

In [ ]:
def _harmonic_location(n, _=None):
    touch_points = []
    m = 1 / (1 - 1/n)
    for i in range(1, n):
        touch_points.append(12 * np.log2(1 / (1 - 1/n) * i))

    return touch_points

In [ ]:
_harmonic_location(2)

In [ ]:
def harmonic_location(n, note, microtones, up=True):
    distances = _harmonic_location(n, up)
    midi = note_name_to_midi(note)
    positions = [d + midi for d in distances]
    pitches = [float_midi_to_microtonal_pitch(p, microtones) for p in positions]

    return pitches

In [ ]:
def note_object_to_midi(n):
    base = note_name_to_midi(n.name.split(' / ')[0]) % 12
    return base + n.cents_deviation/100.

In [ ]:
note = 'C4'

locations_of_all_harmonics = []
for n in range(2,13):
    locations_of_nth_harmonic = harmonic_location(n, note, 8)
    locations_of_all_harmonics.append(locations_of_nth_harmonic)

In [ ]:
# remove all nodes that appear in a previous harmonic
for idx, loc_nth_harm in enumerate(reversed(locations_of_all_harmonics)):
    for item in loc_nth_harm:
        if any(item in lnh for lnh in locations_of_all_harmonics[idx:]):
            loc_nth_harm.remove(item)

In [ ]:
locations_of_all_harmonics[2], locations_of_all_harmonics[1]

In [ ]:
note_object_to_midi(my_overtones[5])

In [ ]:
for note in ["c3", "g3", "d4", "a4", "e5"]:
    harm_locs = []
    
    for i in range(2, 13):
        _locs = harmonic_location(i, note, 8)
        locs = []
        for n in _locs:
            if not any(n in l for l in harm_locs):
                locs.append(n)
        harm_locs.append(locs)
    
    anan = []
    for hit in harm_locs:
        ebenhams = []
        for mt in hit:
            mt_value = note_object_to_midi(mt)
            for nnn in my_overtones:
                nnn_value = note_object_to_midi(nnn)
                if abs(mt_value - nnn_value) <= 0.14:
                    octaveless_ebenhams_value_pairs = [(note_object_to_midi(a), note_object_to_midi(b)) for (a, b) in ebenhams]
                    if (nnn_value, mt_value) not in octaveless_ebenhams_value_pairs:
                        ebenhams.append((nnn, mt))
        anan.extend(ebenhams)
    
    
    touch_screen = sorted(anan, key=lambda x: note_name_to_midi(x[1].name.split(' / ')[0]) + x[1].microtone_deviation)
    # harman = [float_midi_to_microtonal_pitch(overtone(note_name_to_midi(note), x[0]), 8) for x in touch_screen]
    
    print(note)
    for a, b in zip(touch_screen, harman):
        print(a[0], a[1], '  = ', b)

In [ ]:
for i in range(2, 13):
    print(i, harmonic_location(i, note, 8), "    ", float_midi_to_microtonal_pitch(overtone(note_name_to_midi(note), i), 8))

In [ ]:
def harmonic_reduce_octave(n):
    while n % 2 == 0:
        n = n // 2

    return n

In [ ]:
def frac_harm_reduce_octave(f):
    a, b = f.numerator, f.denominator
    a = harmonic_reduce_octave(a)
    b = harmonic_reduce_octave(b)

    return Fraction(a, b)

In [ ]:
def arithmetic_mean(*xs):
    xs = [Fraction(x) for x in xs]
    return frac_harm_reduce_octave(Fraction(sum(xs), len(xs)))

In [ ]:
def harmonic_mean(*xs):
    xs = [Fraction(x) for x in xs]
    return frac_harm_reduce_octave(Fraction(len(xs), sum(1/x for x in xs)))

In [ ]:
def divide_into(a, b, divisor):
    if a > b:
        a, b = b, a

    diff = b - a
    step = Fraction(diff / divisor)

    return [frac_harm_reduce_octave(a + step * i) for i in range(divisor + 1)]

In [ ]:
# can also use difference tone (maybe sum tone?)
def get_all_related_overtones(*xs, divisor=None):
    am = arithmetic_mean(*xs)
    hm = harmonic_mean(*xs)

    ams = f'Arithmetic mean: {am}'
    hms = f'Harmonic mean: {hm}'

    if len(xs) == 2 and divisor is not None:
        division = divide_into(*xs, divisor)
        division_s = f'Division: {[str(d) for d in division]}'
        return ams, hms, division_s, frac_harm_reduce_octave(sum(xs)), frac_harm_reduce_octave(abs(xs[0]-xs[1]))

    return ams, hms

In [ ]:
get_all_related_overtones(1, 19, divisor=4), get_all_related_overtones(3, 13, divisor=4)

In [ ]:
get_all_related_overtones(5, 11, divisor=4), get_all_related_overtones(7, 19, divisor=4)

In [ ]:
get_all_related_overtones(5, 19, divisor=4), get_all_related_overtones(7, 11, divisor=4)

In [ ]:
get_all_related_overtones(7, 13, divisor=4), get_all_related_overtones(1, 5, divisor=4)

In [ ]:
get_all_related_overtones(3, 19, divisor=4), get_all_related_overtones(1, 13, divisor=4)

In [ ]:
get_all_related_overtones(7, 19, divisor=4), get_all_related_overtones(3, 17, divisor=4)

In [ ]:
get_all_related_overtones(11, 19, divisor=4), get_all_related_overtones(13, 17, divisor=4)

In [ ]:
get_all_related_overtones(7, 5, divisor=4), get_all_related_overtones(1, 13, divisor=4)

In [ ]:
sorted([1, 3, 19, 13, 5, 11, 7, 17])

In [ ]:
[reduce_octaves(hz_to_midi(f0 * midi_to_hz(note_name_to_midi('D4')))) - 2 for f0 in [1, 3, 5, 7, 11, 13, 17, 19]]